# 组合优化

组合构建本身并不引入新的资产，也没有对收益率的预测产生更新的观点，它的核心是投资的分散化和风险的控制。最早的组合构建可以追溯到上世纪 50 年代的 Markowiz 均值方差理论。

## 约束条件类型

大多数的组合构建最终都归结为满足一定约束条件的最优化问题。不同的组合构建问题的优化目标各不一样，但常见的约束条件基本属于以下的范畴。

**权重约束条件**

$$
\mathbf{a}\le\mathbf{w}\le\mathbf{b},\ w_i\in [a_i, b_i], k=1,\ldots,n
$$
其中，$\mathbf{a}$ 是权重约束下限，$\mathbf{b}$ 是权重约束上限，比如 $\mathbf{w}\ge \mathbf{0}$ 表示限制为多头投资。

**预算约束条件**
$$
\mathbf{1}^T\cdot\mathbf{w}=a
$$
其中，$\mathbf{1}$ 是全 1 向量，$a$ 是总权重，比如 $\mathbf{1}^T\cdot\mathbf{w}=1$ 表示限制为全额投资。

**因子暴露约束条件**
$$
\mathbf{x}^T\cdot\left(\mathbf{w}-\mathbf{w}_b\right)=a
$$
其中，$\mathbf{x}$ 是因子暴露向量，$\mathbf{w}_b$ 是基准权重向量，$a$ 是组合相对于基准的超额因子暴露，比如 $\mathbf{x}^T\cdot \left(\mathbf{w}-\mathbf{w}_b\right)=0$ 表示风格中性。

**波动率约束条件**
$$
\left(\mathbf{w}-\mathbf{w}_b\right)^T\cdot \mathbf{\Sigma}\cdot \left(\mathbf{w}-\mathbf{w}_b\right)\le \sigma^2
$$
其中，$\mathbf{\Sigma}$ 是收益率的预期协方差矩阵，$\mathbf{w}_b$ 是基准权重向量，$\sigma$ 是组合相对于基准的跟踪误差限制。如果 $\mathbf{w}_b=\mathbf{0}$ 则是对于多头波动率的控制。

**换手率约束条件**
$$
\begin{align}
    & \sum\limits_{i=1}^n{\left| {{w}_{i}}-{{w}_{0i}} \right|}\le a \\
    & \sum\limits_{i=1}^n{\left( {{w}_{i}}-{{w}_{0i}} \right)^{+}}\le a \\
    & \sum\limits_{i=1}^n{\left( {{w}_{i}}-{{w}_{0i}} \right)^{-}}\le a \\
    & A_i\cdot\left|w_i-w_{0i}\right|\le a_i \\
    & A_i\cdot\left(w_i-w_{0i}\right)^+\le a_i \\
    & A_i\cdot\left(w_i-w_{0i}\right)^-\le a_i \\
\end{align}
$$
其中，前三个条件是总换手约束、总买入换手约束以及总卖出换手约束，后面三个条件分别是个券换手约束、个券买入换手约束以及个券卖出换手约束，$w_{0i}$ 是当前持有的资产 $i$ 的权重，$A_i$ 是资产 $i$ 的成交额。

**预期收益约束条件**
$$
\mathbf{\mu}^T\cdot\mathbf{w}\ge a
$$
其中，$\mathbf{\mu}$ 是预期收益率向量，$a$ 是预期收益下限。

**非零权重数目约束条件**
$$
\operatorname{nnz}\left( \mathbf{w} \right)\le N
$$
其中，$\operatorname{nnz}$ 是取非零个数的函数，$N$ 是非零权重数目上限。

## 均值方差模型

均值方差模型来源于上世纪 50 年代的 Markowiz 均值方差理论，即在给定约束条件下期望效用最大化。其导出的优化问题的一般形式如下：
$$
\begin{align}
  & \underset{\mathbf{w}}{\mathop{\max }}\,\left\{\gamma\cdot\mathbf{\mu}^T\cdot\mathbf{w}-\frac{\lambda }{2}\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}-\operatorname{TC}\left( \mathbf{w} \right) \right\} \\ 
  & \operatorname{TC}\left( \mathbf{w} \right)={\lambda_1}\sum\limits_{i=1}^n{\left| {{w}_{i}}-{{w}_{0i}} \right|} + {\lambda_2}\sum\limits_{i=1}^n{\left( {{w}_{i}}-{{w}_{0i}} \right)^{+}} + {\lambda_3}\sum\limits_{i=1}^n{{\left( {{w}_{i}}-{{w}_{0i}} \right)}^{-}} \\
  & s.t.\ \mathbf{w}\in\mathfrak{C}
\end{align}
$$
其中，$\mathbf{w}$ 是组合权重，优化变量，$\mathbf{\mu}$ 是预期收益率，$\mathbf{\Sigma}$ 是收益率的预期协方差矩阵，$\gamma$ 是收益项系数，$\lambda\ge 0$ 是风险厌恶系数，$\operatorname{TC}$ 是交易成本惩罚函数，$\lambda_1\ge 0$ 是买卖交易费率，$\lambda_2\ge 0$ 是买入交易费率，$\lambda_3\ge 0$ 是卖出交易费率，$\mathbf{w}_0$ 是当前持有的组合权重，$\mathfrak{C}$ 是上一节中提到的约束条件自由组合成的约束集。

均值方差模型导出的优化问题通常是一个凸规划问题（如果含有非零权重数目约束条件，则为整数规划问题），在某些参数取值特殊的情况下可以退化为二次规划或者线性规划问题，很多成熟的凸优化算法都可以有效的求解该问题。

下面以最小方差模型作为示例:

In [1]:
import time
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS

if __name__=='__main__':
    HDB = QS.FactorDB.HDF5DB()
    HDB.connect()
    RDB = QS.RiskModel.HDF5FRDB()
    RDB.connect()
    
    CFT = QS.FactorDB.CustomFT("CFT")
    FT = HDB.getTable("ElementaryFactor")
    CFT.addFactors(factor_table=FT, factor_names=["月收益率", "是否在市", "特殊处理", "涨跌停", "交易状态", "成交金额"])
    CFT.addFactors(factor_table=HDB.getTable("IndexConstituentFactor"), factor_names=["沪深300成份权重", "中证500成份权重", "上证50成份权重"])
    CFT.addFactors(factor_table=HDB.getTable("BarraDescriptor"), factor_names=["Industry"])
    CFT.addFactors(factor_table=HDB.getTable("BarraFactor"), factor_names=None)
    CFT.setID(FT.getID(ifactor_name="月收益率"))
    CFT.setDateTime(FT.getDateTime(ifactor_name="月收益率"))
    
    # 创建投资组合构造器
    import matlab.engine
    EngList = matlab.engine.find_matlab()
    if EngList:
        MatlabEng = matlab.engine.connect_matlab(name=EngList[0])
    else:
        MatlabEng = None
    PC = QS.PortfolioConstructor.MatlabPC(matlab_eng=MatlabEng)
    
    # 设置相关数据
    TargetDT = dt.datetime(2018, 8, 31, 23, 59, 59, 999999)
    TargetIDs = CFT.getID(idt=TargetDT)[:500]
    PC["目标ID"] = TargetIDs
    PC["预期收益"] = CFT.readData(factor_names=["月收益率"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
    #PC["协方差矩阵"] = RDB.readCov("BarraRiskData", dts=[TargetDT], ids=TargetIDs)[TargetDT]
    PC["因子协方差阵"] = RDB.readFactorCov("BarraRiskData", dts=[TargetDT]).loc[TargetDT]
    PC["风险因子值"] = RDB.readFactorData("BarraRiskData", dts=[TargetDT], ids=TargetIDs).loc[:, TargetDT, :]
    PC["特异性风险"] = RDB.readSpecificRisk("BarraRiskData", dts=[TargetDT], ids=TargetIDs).loc[TargetDT, :]
    PC["因子暴露"] = CFT.readData(factor_names=["Size"], ids=TargetIDs, dts=[TargetDT]).iloc[:, 0, :]
    PC["成交金额"] = CFT.readData(factor_names=["成交金额"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
    PC["初始投资组合"] = pd.Series(0.0,index=TargetIDs)
    PC["基准投资组合"] = CFT.readData(factor_names=["沪深300成份权重"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
    PC["总财富"] = 1000000000
    
    # 设置优化目标
    Objective = QS.PortfolioConstructor.MeanVarianceObjective(pc=PC)
    Objective["收益项系数"] = 0.0
    Objective["风险厌恶系数"] = 1.0
    PC["优化目标"] = Objective
    
    # 设置约束条件
    # 预算约束
    iConstraint = QS.PortfolioConstructor.BudgetConstraint(pc=PC)
    iConstraint["限制上限"] = 1.0
    iConstraint["限制下限"] = 1.0
    PC["约束条件"].append(iConstraint)
    # 权重约束
    iConstraint = QS.PortfolioConstructor.WeightConstraint(pc=PC)
    iConstraint["限制上限"] = 1.0
    iConstraint["限制下限"] = 0.0
    PC["约束条件"].append(iConstraint)
    
    # 求解优化问题
    StartT = time.clock()
    Portfolio, ResultInfo = PC.solve()
    print("计算耗时: %.2f" % (time.clock()-StartT, ))
    print(Portfolio)

计算耗时: 4.28
000001.SZ    7.117370e-03
000002.SZ    1.627233e-10
000004.SZ    6.402087e-10
000005.SZ    1.902520e-10
000006.SZ    1.173289e-10
000007.SZ    4.148474e-10
000008.SZ    1.961924e-10
000009.SZ    3.137823e-10
000010.SZ    3.390528e-10
000011.SZ    1.138121e-10
000012.SZ    1.914318e-10
000014.SZ    8.413458e-11
000016.SZ    2.412963e-10
000017.SZ    1.164719e-10
000018.SZ    1.064048e-10
000019.SZ    1.103550e-10
000020.SZ    1.398957e-10
000021.SZ    5.882847e-10
000022.SZ    4.079393e-10
000023.SZ    6.988066e-11
000025.SZ    3.927398e-10
000026.SZ    3.099312e-10
000027.SZ    3.005057e-09
000028.SZ    3.649939e-10
000029.SZ    1.693427e-01
000030.SZ    5.622226e-09
000031.SZ    1.612461e-10
000032.SZ    2.188266e-10
000034.SZ    1.968960e-10
000035.SZ    1.563721e-10
                 ...     
000957.SZ    1.166446e-10
000958.SZ    1.521527e-10
000959.SZ    2.032515e-10
000960.SZ    2.787994e-10
000961.SZ    1.573187e-10
000962.SZ    1.504824e-10
000963.SZ    1.781384e-02
0

## 最大夏普率模型

最大夏普率模型导出的优化问题形式如下：
$$
\begin{align}
& \underset{\mathbf{w}}{\mathop{\max }}\,\frac{\mathbf{\mu}^T\cdot\mathbf{w}}{\sqrt{\mathbf{w}^T\Sigma\mathbf{w}}}\\ 
& s.t.\ \mathbf{w}\in\mathfrak{C}
\end{align}
$$
其中，$\mathbf{w}$ 是组合权重，优化变量，$\mathbf{\mu}$ 是预期收益率，$\mathbf{\Sigma}$ 是收益率的预期协方差矩阵，$\mathfrak{C}$ 是上一节中提到的 $约束条件$_ 自由组合成的约束集。

上述优化问题的目标函数是有理分式的形式，通常将其转化成凸规划和一维搜索的组合问题。

对于任意的预期收益水平 $x$，定义一元函数 $f\left(x\right)$ 为如下凸规划问题的最优解：
$$
\begin{align}
& f\left(x\right) = \underset{\mathbf{w}}{\mathop{\max }}\,\frac{x}{\sqrt{\mathbf{w}^T\Sigma\mathbf{w}}}\\ 
& s.t.\ \mathbf{w}\in\mathfrak{C},\ \mathbf{\mu}^T\cdot\mathbf{w}=x
\end{align}
$$
对于 $f\left(x\right)$，求其极大值，对应的最优解为 $x^*$，对应该预期收益水平 $x^*$ 的凸规划问题的解 $w^*$ 就是原问题的解。

## 风险预算模型

对于组合 $\mathbf{w}$ 的风险度量 $\mathcal{R}\left(\mathbf{w}\right)$ 采用波动率度量：
$$
\mathcal{R}\left(\mathbf{w}\right) = \sigma\left(\mathbf{w}\right) = \sqrt{\mathbf{w}^T\cdot\mathbf{\Sigma}\cdot\mathbf{w}}
$$
其中，$\Sigma$ 是协方差矩阵，边际风险贡献定义为 $\sigma\left(\mathbf{w}\right)$ 相对于 $\mathbf{w}$ 的梯度：
$$
\nabla_{\mathbf{w}}\sigma\left(\mathbf{w}\right) = \frac{\mathbf{\Sigma}\mathbf{w}}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}
$$
第 $i$ 个证券对于组合的风险贡献度定义为：
$$
\mathcal{RC}_i=w_i\cdot\frac{\left(\mathbf{\Sigma}\mathbf{w}\right)_i}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}
$$
则可以将组合的总风险分解到各个证券上：
$$
\begin{align}
\sigma\left(\mathbf{w}\right) &= \mathbf{w}^T\cdot\frac{\mathbf{\Sigma}\mathbf{w}}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}\\
&= \sum\limits_{i=1}^{n}w_i\cdot\frac{\left(\mathbf{\Sigma}\mathbf{w}\right)_i}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}\\
&= \sum\limits_{i=1}^{n}\mathcal{RC}_i\\
\end{align}
$$
对于给定的风险预算 $\mathbf{b}$，定义风险预算组合为：
$$
\left\{ \begin{align}
           & \mathcal{RC}_i=b_i\cdot\mathcal{R}\left(\mathbf{w}\right)  \\
           & b_i\ge0  \\
           & w_i\ge0  \\
           & \sum\limits_{i=1}^{n}b_i = 1  \\
           & \sum\limits_{i=1}^{n}w_i = 1  \\
        \end{align}
\right.
$$
当 $b_i=\frac{1}{n},i=1,\ldots,n$ 时该模型即为风险平价模型，即各个资产的风险贡献度相同。

该非线性方程组的求解将转化为如下的优化问题：
$$
\begin{align}
  & \underset{\mathbf{w}}{\mathop{\min}}\,f\left(\mathbf{w};\mathbf{b}\right)=\sum\limits_{i=1}^{n}\left(\mathcal{RC}_i-b_i\mathcal{R}(\mathbf{w})\right)^2 \\ 
 & s.t.\ \mathbf{1}^T\cdot\mathbf{w}=1,\mathbf{0}\le\mathbf{w}\le\mathbf{1} \\ 
\end{align}
$$
或者：
$$
\begin{align}
  & \underset{\mathbf{w}}{\mathop{\min}}\,f\left(\mathbf{w};\mathbf{b}\right)=\sum\limits_{i=1}^{n}\sum\limits_{j=1}^{n}{\left(\frac{\mathcal{RC}_i}{b_i}-\frac{\mathcal{RC}_j}{b_j}\right)^2} \\ 
 & s.t.\ \mathbf{1}^T\cdot\mathbf{w}=1,\mathbf{0}\le\mathbf{w}\le\mathbf{1} \\ 
\end{align}
$$
以下主要针对第二个优化问题构建求解方法。

通过计算目标函数 $f\left(\mathbf{w};\mathbf{b}\right)$ 的梯度和 Hessian 矩阵可以帮助减少计算次数，尤其在资产数目很大的时候，目标函数的梯度为：
$$
\begin{align}
    \frac{\partial f}{\partial w_k} &= \frac{4n}{\sigma^2}w_k\frac{\left(\Sigma \mathbf{w}\right)^2_k}{b_k^2} - \frac{4}{\sigma^2}\frac{\left(\Sigma\mathbf{w}\right)_k}{b_k}\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\\
    &+ \frac{4n}{\sigma^2}\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i^2}\Sigma_{ik} - \frac{4}{\sigma^2}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\cdot\left(\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{ik}\right)\\
    &- \frac{4n}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)^2_i}{b_i^2}\right)\cdot\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right) + \frac{4}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)^2\cdot\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right)\\
\end{align}
$$
矩阵形式为：（矩阵之间对应元素相乘记为 $.*$，矩阵的乘方也是各个元素分别乘方）
$$
\begin{align}
    \frac{\partial f}{\partial\mathbf{w}} &= \frac{4n}{\sigma^2}w.*\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2} - \frac{4}{\sigma^2}\left(\mathbf{w}^T\left((\Sigma\mathbf{w}).*\mathbf{b}^{-1}\right)\right)\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\\
    &+ \frac{4n}{\sigma^2}\Sigma\cdot\left(\mathbf{w}^2.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right) - \frac{4}{\sigma^2}\left(\mathbf{w}^T\left((\Sigma\mathbf{w}).*\mathbf{b}^{-1}\right)\right)\cdot\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\\
    &- \frac{4n}{\sigma^4}\left(\mathbf{w}^2\right)^T\cdot\left((\Sigma\mathbf{w})^2.*\mathbf{b}^{-2}\right)\cdot\Sigma\mathbf{w} + \frac{4}{\sigma^4}\left(\mathbf{w}^T\left((\Sigma\mathbf{w}).*\mathbf{b}^{-1}\right)\right)^2\cdot\Sigma\mathbf{w}\\
\end{align}
$$
目标函数的 Hessian 矩阵为：
$$
\begin{align}
    \frac{\partial^2f}{\partial w_k\partial w_l} &= -\frac{2}{\sigma^2}\frac{\partial f}{\partial w_k}\left(\Sigma\mathbf{w}\right)_l + \frac{4n}{\sigma^2}\left[\delta_{kl}\frac{\left(\Sigma\mathbf{w}\right)^2_k}{b_k^2}+2w_k\frac{\left(\Sigma\mathbf{w}\right)_k}{b_k^2}\Sigma_{kl}\right]\\
    &- \frac{4}{\sigma^2}\left[\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l}+\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{il}\right]\frac{\left(\Sigma\mathbf{w}\right)_k}{b_k} - \frac{4}{\sigma^2}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\frac{\Sigma_{kl}}{b_k}\\
    &+ \frac{4n}{\sigma^2}\left[2w_l\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l^2}\Sigma_{lk}+\sum\limits_{i=1}^{n}\frac{w^2_i}{b_i^2}\Sigma_{ik}\Sigma_{il}\right]-\frac{4}{\sigma^2}\left[\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l}+\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{il}\right]\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{ik}\\
    &- \frac{4}{\sigma^2}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\frac{\Sigma_{lk}}{b_l} - \frac{8n}{\sigma^4}\left[w_l\frac{\left(\Sigma\mathbf{w}\right)_l^2}{b^2_l}+\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)_i}{b^2_i}\Sigma_{il}\right]\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right)\\
    &- \frac{4n}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)^2_i}{b^2_i}\right)\Sigma_{lk} + \frac{8n}{\sigma^6}\left(\Sigma\mathbf{w}\right)_l\left(\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)_i^2}{b_i^2}\right)\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right)\\
    &+ \frac{8}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\left[\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l}+\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{il}\right]\cdot\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\\
    &+ \frac{4}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)^2\cdot\Sigma_{lk} - \frac{8}{\sigma^6}\left(\Sigma\mathbf{w}\right)_l\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)^2\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\\
\end{align}
$$
其中，$\delta_{kl}$ 为 Kronecker 函数，即：
$$
\delta_{kl} = \left\{\begin{matrix}
                     1,k=l  \\
                     0,k\ne l  \\
                     \end{matrix} \right.
$$
矩阵形式为：（矩阵之间对应元素相乘记为 $.*$，矩阵的乘方也是各个元素分别乘方）
$$
\begin{align}
    \frac{\partial^2f}{\partial\mathbf{w}^2} &= -\frac{2}{\sigma^2}\frac{\partial f}{\partial w}\cdot \left(\Sigma\mathbf{w}\right)^T + \frac{4n}{\sigma^2}\operatorname{D}\left(\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)\\
    &+ \frac{8n}{\sigma^2}\operatorname{D}\left(\mathbf{w}.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right)\cdot\Sigma - \frac{4}{\sigma^2}\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)^T\\
    &- \frac{4}{\sigma^2}\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\left(\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)^T - \frac{4}{\sigma^2}\left(\mathbf{w}^T\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\operatorname{D}\left(\mathbf{b}^{-1}\right)\cdot\Sigma\\
    &+ \frac{8n}{\sigma^2}\Sigma\cdot\operatorname{D}\left(\mathbf{w}.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right) + \frac{4n}{\sigma^2}\Sigma\cdot\operatorname{D}\left(\mathbf{w}^2.*\mathbf{b}^{-2}\right)\cdot\Sigma\\
    &- \frac{4}{\sigma^2}\left(\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)^T - \frac{4}{\sigma^2}\left(\Sigma\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)\cdot\left(\Sigma\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)^T\\
    &- \frac{4}{\sigma^2}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\Sigma\cdot\operatorname{D}\left(\mathbf{b}^{-1}\right) - \frac{8n}{\sigma^4}\left(\Sigma\mathbf{w}\right)\cdot\left(\mathbf{w}.*\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)^T\\
    &- \frac{8n}{\sigma^4}\left(\Sigma\mathbf{w}\right)\left(\Sigma\left(\mathbf{w}^2.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right)\right)^T - \frac{4N}{\sigma^4}\left(\left(\mathbf{w}^2\right)^T\cdot\left(\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)\right)\cdot\Sigma\\
    &+ \frac{8n}{\sigma^6}\left(\left(\mathbf{w}^2\right)^T\cdot\left(\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)\right)\left(\Sigma\mathbf{w}\right)\cdot\left(\Sigma\mathbf{w}\right)^T\\
    &+ \frac{8}{\sigma^4}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\left(\Sigma\mathbf{w}\right)\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)^T\\
    &+ \frac{8}{\sigma^4}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\left(\Sigma\mathbf{w}\right)\cdot\left(\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)^T\\
    &+ \frac{4}{\sigma^4}\left(\mathbf{w}^T\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)^2\Sigma - \frac{8}{\sigma^6}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)^2\cdot\left(\Sigma\mathbf{w}\right)\cdot\left(\Sigma\mathbf{w}\right)^T\\
\end{align}
$$
其中，$\operatorname{D}\left(\cdot\right)$ 是对角化函数，即对于向量 $\mathbf{x}$，有：
$$
\operatorname{D}\left(\mathbf{x}\right) = \left(\begin{matrix}
                                                   {{x}_{1}} & 0 & \cdots  & 0  \\
                                                   0 & {{x}_{2}} & \cdots  & 0  \\
                                                   \vdots  & \vdots  & \ddots  & \vdots   \\
                                                   0 & 0 & \cdots  & {{x}_{n}}  \\
                                                \end{matrix}\right)
$$
对于维度较高的问题，也可以先求解如下的凸规划问题：
$$
\begin{align}
& \underset{\mathbf{x}}{\mathop{\min}}\,\mathcal{R}\left(\mathbf{x}\right)=\sqrt{\mathbf{x}^T\Sigma\mathbf{x}}\\
& s.t.\ \sum\limits_{i=1}^{n}b_i\operatorname{ln}x_i\ge c, \mathbf{0}\le\mathbf{x}\le\mathbf{1}\\
\end{align}
$$
其中，$c$ 是满足 $c<\sum\limits_{i=1}^{n}b_i\operatorname{ln}b_i$ 的任意常数，最后将最优解归一化得到原问题的解：
$$
\mathbf{w} = \frac{\mathbf{x}}{\mathbf{1}^T\mathbf{x}}
$$

## 最大分散化模型

该模型试图构造分散程度最大的投资组合，分散程度通过比较组合整体波动率和组合中资产的加权平均波动率的距离来度量，该模型可归结为以下的优化问题
$$
\begin{align}
  & \underset{\mathbf{w}}{\mathop{\max}}\,\frac{\sum\limits_{i=1}^{n}w_i\sigma_i}{\sqrt{\mathbf{w}^T\Sigma\mathbf{w}}}\\
 & s.t.\ \mathbf{1}^T\mathbf{w}=1, \mathbf{0}\le\mathbf{w}\le\mathbf{1}\\
\end{align}
$$
其中，$\sigma_i$ 是资产 $i$ 的波动率，分母是组合的波动率，分子是组合中资产的加权平均波动率，也可以看成所有资产的相关系数为 1 时的波动率，即分散化完全无效时的情形。分子和分母的本质差别在于相关性，为了最大化这个比值，分母中包含的相关性必须被最小化。所以该配置模型试图选择能够最小化标的资产间相关性的资产，从而达到所谓的最大化分散的目的。

从计算的角度看，目标函数为有理式，但可以转换成二次规划问题。对协方差矩阵进行分解：
$$
\mathbf{\Sigma} = \mathbf{D}^{1/2}\cdot\mathbf{P}\cdot\mathbf{D}^{1/2}
$$
其中，$\mathbf{D}$ 是取协方差阵的对角元素形成的对角阵，而 $\mathbf{P}$ 是相关系数矩阵，则容易证明原问题可以按照下列步骤求解。
1. 求解二次规划问题：
$$
\begin{align}
& \underset{\mathbf{x}}{\mathop{\min}}\,\frac{1}{2}\mathbf{x}^T\cdot\mathbf{P}\cdot\mathbf{x}\\
& s.t.\ \mathbf{1}^T\mathbf{x}=1, \mathbf{0}\le\mathbf{x}\le\mathbf{1}\\
\end{align}
$$
2. 使用资产的波动率调整中间变量 $\mathbf{x}$：
$$
\mathbf{\xi} = \mathbf{D}^{-1/2}\cdot\mathbf{x},\ \xi_i=\frac{x_i}{\sigma_i}
$$
3. 归一化：
$$
\mathbf{w} = \frac{\mathbf{\xi}}{\mathbf{1}^T\mathbf{\xi}},\ w_i=\frac{\xi_i}{\sum\limits_{i=1}^{n}\xi_i}
$$
还可以从计量组合中资产的平均相关水平的角度来理解最大分散化模型。一个组合风险并非简单的将资产的波动率加权平均，同样组合中资产的相关性水平也不能简单的对两两相关系数取平均来衡量，它们中蕴含着非线性的关系。

从组合方差的计算公式入手：
$$
\sigma^2 = \sum\limits_{i=1}^{n}w_i^2\sigma_i^2+2\sum\limits_{i=1}^{n-1}\sum\limits_{j>i}^{n}\rho_{ij}w_iw_j\sigma_i\sigma_j
$$
其中，$\rho_{ij}$ 是资产 $i$ 和 $j$ 的相关系数。

假设 $\bar{\rho}$ 是资产间的平均相关水平，则有：
$$
\sigma^2 = \sum\limits_{i=1}^{n}w_i^2\sigma_i^2+2\sum\limits_{i=1}^{n-1}\sum\limits_{j>i}^{n}\rho_{ij}w_iw_j\sigma_i\sigma_j = \sum\limits_{i=1}^{n}w_i^2\sigma_i^2+2\sum\limits_{i=1}^{n-1}\sum\limits_{j>i}^{n}\bar{\rho}w_iw_j\sigma_i\sigma_j
$$
定义组合加权相关水平（WPC）指标为 $\bar{\rho}$：
$$
WPC = \bar{\rho} = \frac{\sum\limits_{i=1}^{n-1}\sum\limits_{j>i}^{n}\rho_{ij}w_iw_j\sigma_i\sigma_j}{\sum\limits_{i=1}^{n-1}\sum\limits_{j>i}^{n}w_iw_j\sigma_i\sigma_j}
$$
本质上，WPC 是经过资产波动率调整的加权平均两两相关系数。

一个自然的想法是 WPC 较低的组合应该是更充分分散的组合。再引入两个概念——分散度比率（DR）和集中度比率（CR）：
$$
\begin{align}
    & DR = \frac{\sum\limits_{i=1}^{n}w_i\sigma_i}{\sigma}\\
    & CR = \frac{\sum\limits_{i=1}^{n}w_i^2\sigma^2_i}{\left(\sum\limits_{i=1}^{n}w_i\sigma_i\right)^2}\\
\end{align}
$$
分散度比率（DR）即是前面最大化分散模型的优化目标，而集中度比率（CR）是仅仅只考虑资产波动率的组合集中度的简单度量。一个完全集中的多头组合是将所有的资金投资在单一资产上，此时 $CR=1$，而波动率倒数加权的投资组合具有最低的集中度，此时 $CR=\frac{1}{n}$。

上述三个指标有一个简单的公式联系在一起：
$$
DR = \frac{1}{\sqrt{WPC\cdot\left(1-CR\right)+CR}} = \frac{1}{\sqrt{CR\cdot\left(1-WPC\right)+WPC}}
$$
从而最大化分散度比率（DR）可以通过降低集中度比率（CR）或者降低组合的平均相关水平（WPC）来实现。

从上面的论述可知，波动率倒数组合是最小集中度组合，也称为波动率平价组合，而最大分散化组合也被称为相关性平价组合。